In [1]:
pip install tensorflow opencv-python scikit-learn matplotlib


Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import zipfile
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [5]:
zip_path = "chest.zip"
extract_path = "chest"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [7]:
# Step 5: Load and preprocess data
def load_data(data_dir, img_size=224):
    categories = ['Normal', 'Tuberculosis']
    data = []
    for label, category in enumerate(categories):
        path = os.path.join(data_dir, category)
        for img_file in os.listdir(path):
            try:
                img_path = os.path.join(path, img_file)
                img = cv2.imread(img_path)
                img = cv2.resize(img, (img_size, img_size))
                data.append([img, label])
            except Exception as e:
                continue
    return data

data_dir = "chest/chest"
data = load_data(data_dir)
print(f"Total images loaded: {len(data)}")

Total images loaded: 4200


In [9]:
# Step 6: Shuffle and prepare dataset
import numpy as np
np.random.shuffle(data)
X, y = zip(*data)
X = np.array(X) / 255.0  # Normalize
y = to_categorical(np.array(y), num_classes=2)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)


In [11]:
# Step 7: Define CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


C:\Users\dasar\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 222, 222, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 109, 109, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 52, 52, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,170,114 (42.61 MB)

 Trainable params: 11,169,666 (42.61 MB)

 Non-trainable params: 448 (1.75 KB)

In [13]:
# Step 8: Data augmentation and training
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rotation_range=10, zoom_range=0.1, horizontal_flip=True)
datagen.fit(X_train)

history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    epochs=10,
                    validation_data=(X_val, y_val))


C:\Users\dasar\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 148s 1s/step - accuracy: 0.8874 - loss: 2.9737 - val_accuracy: 0.1905 - val_loss: 20.5310
Epoch 2/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 137s 1s/step - accuracy: 0.9442 - loss: 0.6901 - val_accuracy: 0.5476 - val_loss: 2.5229
Epoch 3/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 138s 1s/step - accuracy: 0.9572 - loss: 0.2711 - val_accuracy: 0.9155 - val_loss: 1.5771
Epoch 4/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 137s 1s/step - accuracy: 0.9431 - loss: 0.5054 - val_accuracy: 0.8607 - val_loss: 0.6876
Epoch 5/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 673s 6s/step - accuracy: 0.9564 - loss: 0.1920 - val_accuracy: 0.9393 - val_loss: 0.7162
Epoch 6/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 137s 1s/step - accuracy: 0.9681 - loss: 0.2208 - val_accuracy: 0.9310 - val_loss: 1.3132
Epoch 7/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 137s 1s/step - accuracy: 0.9564 - loss: 0.2134 - val_accuracy: 0.8905 - val_loss: 2.3266
Epoch 8/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 137s 1s/step - accuracy: 0.9616 - loss: 0.1724 - val_acc

In [15]:
# Step 9: Evaluate model
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")

y_pred = np.argmax(model.predict(X_val), axis=1)
y_true = np.argmax(y_val, axis=1)
print(classification_report(y_true, y_pred, target_names=['Normal', 'Tuberculosis']))


27/27 ━━━━━━━━━━━━━━━━━━━━ 5s 181ms/step - accuracy: 0.8966 - loss: 1.2525
Validation Accuracy: 91.07%
27/27 ━━━━━━━━━━━━━━━━━━━━ 5s 182ms/step
              precision    recall  f1-score   support

      Normal       0.90      1.00      0.95       700
Tuberculosis       1.00      0.46      0.63       140

    accuracy                           0.91       840
   macro avg       0.95      0.73      0.79       840
weighted avg       0.92      0.91      0.90       840



In [17]:
# Step 10: Save the model
model.save("chest_xray_model.h5")
print("Model saved as chest_xray_model.h5")

Model saved as chest_xray_model.h5


In [23]:
# Step 11: Real-time prediction function
def diagnose_image(img_path):
    img = cv2.imread(img_path)
    if img is None:
        print("Error: Image not found or path is incorrect.")
        return None
    img = cv2.resize(img, (224, 224))
    img = np.expand_dims(img / 255.0, axis=0)
    pred = model.predict(img)
    label = np.argmax(pred)
    confidence = np.max(pred)
    return ("Normal" if label == 0 else "Tuberculosis", f"{confidence * 100:.2f}%")

# Call the function here
result, confidence = diagnose_image('i1.jpeg')
print("Prediction:", result)
print("Confidence:", confidence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
Prediction: Normal
Confidence: 100.00%
